**Домашнее задание 3. Разрешение семантической многозначности.**

Описание задания:
Выберите одну из предложенных лексем: свет, острый, лук, замок.

1. Выберите 2-3 значения лексемы по словарю (рекомендую использовать МАС, обязательно напишите, каким словарем пользовались). Для каждого значения придумайте несколько "диагностических" слов (которые сигнализируют о реализации одного из значений в контексте): например, ключ — вода / дверь. Подберите по НКРЯ контексты для выбранной лексемы, содержащие диагностические слова. Таким образом, получается размеченный тестсет.

2. Для каждого контекста определите значение автоматически — с помощью AdaGram / ELMo на выбор. В случае с AdaGram вы сразу получаете какой-то номер значения, с ELMo — нужно кластеризовать.

3. Возьмите словарные толкования для каждого значения и припишите ему значение автоматически тем же способом, что и в п.2.

4. Придумайте и реализуйте способ соотнести толкования со значениями с помощью тезауруса (Wiki_ru_wordnet / РуТез).

5. Оцените accuracy — в какой доле контекстов автоматически определенно значение для контекста совпадает с автоматически определенным значением толкования.


**1.Создание тестсета**

Из мас были взяты следующие толкования:

ЛУК1, -а (-у), м.

1. Огородное растение сем. лилейных, овощ.
2. собир. Съедобные трубчатые листья или луковицы этого растения. Зеленый лук. Репчатый лук.

Диагностические слова для ЛУК1: лист, зеленый, репчатый.

ЛУК2, -а, м.

1. Ручное оружие для метания стрел в виде дуги, стянутой тетивой. Стрелять из лука.

Диагностические слова для ЛУК2: тетива, стрела, стрелять.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks')

In [0]:
import pandas as pd
import numpy as np

В корпусе 60 примеров контекстов: по 30 для каждого значения.

In [0]:
corpus = pd.read_csv('test_corpus.txt', sep = '.', names = ['text','label'])

In [5]:
corpus.head()

,text,label
0,"Кальмары отварить с морковью, репчатым луком и...",1
1,Кое-кто завёл себе на пирсе маленькие огородик...,1
2,Туда отправляем мелко нарезанными репчатый лук...,1
3,Слегка поджарьте на сковородке нарезанный репч...,1
4,Из другой поездки привез нам мешок репчатого л...,1


**2.Автоматическая разметка контекстов с помощью Adagram**

In [6]:
!wget https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib

--2019-12-09 05:14:47--  https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.32.246
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.32.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1462416741 (1.4G) [application/x-www-form-urlencoded]
Saving to: ‘all.a010.p10.d300.w5.m100.nonorm.slim.joblib’

all.a010.p10.d300.w 100%[===================>]   1.36G  59.3MB/s    in 23s     

2019-12-09 05:15:10 (61.1 MB/s) - ‘all.a010.p10.d300.w5.m100.nonorm.slim.joblib’ saved [1462416741/1462416741]



In [7]:
!pip install git+https://github.com/lopuhin/python-adagram.git

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-6jczy__s
  Running command git clone -q https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-6jczy__s
  Created wheel for adagram: filename=adagram-0.0.1-cp36-cp36m-linux_x86_64.whl size=464607 sha256=be2eadad527898f6b1dacb5749cb8609068e35568b4b7619d85b1aa852c8f4f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-luvlj30h/wheels/11/0f/46/f5df96670df8f7973b4c2311ffc9b02e435a7bd3207f992c4d
Successfully built adagram


In [8]:
!pip install pymystem3==0.1.10

  Found existing installation: pymystem3 0.2.0
    Uninstalling pymystem3-0.2.0:
      Successfully uninstalled pymystem3-0.2.0


In [9]:
pip install pymorphy2

     |████████████████████████████████| 51kB 2.0MB/s 
     |████████████████████████████████| 7.1MB 9.7MB/s 


In [0]:
import adagram

In [0]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

Посмотрим, какие значения для слова "лук" выделяет adagram

In [12]:
vm.word_sense_probs('лук')

[(0, 0.27467387333364635),
 (1, 0.17510064513645113),
 (2, 0.25182447428150506),
 (3, 0.0852764349918363),
 (4, 0.2131232975158767)]

In [75]:
for i in range(5):
  s = []
  for j in vm.sense_neighbors('лук', i):
    s.append(j[0])
  print(f'meaning {i}: {s} ')

/usr/local/lib/python3.6/dist-packages/adagram/model.py:124: RuntimeWarning: invalid value encountered in true_divide
  sim_matrix = np.dot(self.In, s_v) / self.InNorms


meaning 0: ['морковь', 'помидор', 'лук-порей', 'зелень', 'нарезать', 'нашинковывать', 'сельдерей', 'петрушка', 'мелко', 'нарубать'] 
meaning 1: ['арбалет', 'дротик', 'пращ', 'самострел', 'мушкет', 'аркебуза', 'стрелять', 'дробовик', 'пневматический', 'аркебуз'] 
meaning 2: ['чеснок', 'морковь', 'картошка', 'гриб', 'лучок', 'рубленый', 'морковка', 'помидор', 'томат', 'перец'] 
meaning 3: ['лука', 'евангелист', 'иаков', 'лука', 'матфей', 'матфей', 'деяние', 'иоанн', 'апостол', 'ириней'] 
meaning 4: ['лука', 'арбалет', 'колчан', 'тетива', 'стрела', 'копье', 'самострел', 'копье', 'секира', 'палица'] 


Похоже, что значения 0 и 2 относятся к первому выделенному по словарю значению, 1 и 4 - ко второму, а 3 относится к сфере религиозной в качестве имени.

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')
stops = set(stopwords.words('russian'))

def normalize_pm(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

def tokenize(text):
    return token.tokenize(text)

In [16]:
mystem = Mystem()

def disambiguate(model, word, context):
    word, _ = lemmatized_context(word)
    probs = model.disambiguate(word, lemmatized_context(context))
    return 1 + probs.argmax()


def lemmatized_context(s):
    return [w.lower() for w in mystem.lemmatize(" ".join(tokenize(s)))]

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz


In [17]:
corpus['predicted_label'] = [disambiguate(vm, 'лук', context) for context in corpus['text']]
corpus.head()

/usr/local/lib/python3.6/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


,text,label,predicted_label
0,"Кальмары отварить с морковью, репчатым луком и...",1,1
1,Кое-кто завёл себе на пирсе маленькие огородик...,1,3
2,Туда отправляем мелко нарезанными репчатый лук...,1,1
3,Слегка поджарьте на сковородке нарезанный репч...,1,1
4,Из другой поездки привез нам мешок репчатого л...,1,3


**3.Автоматически определим значения для словарных толкований**

Для первого толковаания я оба подпункта объединю в один контекст, так как я их не разделяла при ручной разметке тестового корпуса.

In [0]:
meaning_1 = 'Огородное растение семейства лилейных, овощ. Съедобные трубчатые листья или луковицы этого растения. Зеленый лук. Репчатый лук.'

In [0]:
meaning_2 = 'Ручное оружие для метания стрел в виде дуги, стянутой тетивой. Стрелять из лука.'

In [0]:
vocab_meanings = []
vocab_meanings.append(meaning_1)
vocab_meanings.append(meaning_2)

In [20]:
sense_1 = disambiguate(vm, 'лук', meaning_1)
sense_1

/usr/local/lib/python3.6/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


1

In [21]:
sense_2 = disambiguate(vm, 'лук', meaning_2)
sense_2

/usr/local/lib/python3.6/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


2

**4.Cоотнесение толкования со значениями с помощью Wiki_ru_wordnet**

In [22]:
!pip install wiki_ru_wordnet

     |████████████████████████████████| 20.6MB 1.3MB/s 
  Created wheel for wiki-ru-wordnet: filename=wiki_ru_wordnet-1.0.3-cp36-none-any.whl size=20890443 sha256=0851f0dfd500a2c5236f1f9b8354bdd64e9dbea5231198e87778ac2f76fc1cfa
  Stored in directory: /root/.cache/pip/wheels/48/6a/c8/22758a66c35b21c1491861e8ab4809b1fd466d1d2445e99fda
Successfully built wiki-ru-wordnet


In [0]:
from wiki_ru_wordnet import WikiWordnet

ruwn = WikiWordnet()

In [40]:
for i, j in enumerate(ruwn.get_synsets('лук')):
  words = []
  for w in j.get_words():
    words.append(w.lemma())
  print(i, ', '.join(words))

0 лук
1 лук, луковица
2 лук
3 лук
4 лук
5 лук


In [0]:
synsets = ruwn.get_synsets('лук')

In [45]:
ww_def = []
for i in range(len(synsets)):
  dop = []
  for w in synsets[i].get_words():
    dop.append(w.definition())
    print(f'{i} ', w.definition(), '\n')
  s = ' '.join(dop)
  ww_def.append(s)

0  лук~ru~лук~ru~{{ботан.|ru}} род луковичных [[растений]] (''Allium'') семейства [[Луковые]] (Alliaceae) {{пример|Там какое-то болото было, а {{выдел|лук}} рос на кочках.|Валентин Распутин|Последний срок|1970|источник=НКРЯ}}~65218~74957 

1  лук~ru~лук~ru~{{=|репчатый лук|[[луковица|луковицы]] лука [1] как пищевой продукт}} {{пример|Кроме того, он был сварен с большим количеством {{выдел|лука}}, а Валя не любила варёный {{выдел|лук}}.|Татьяна Тронина|Русалка для интимных встреч|2004|источник=НКРЯ}}~94921~68014 

1  луковица~ru~луковица~ru~головка лука {{пример|Разыскав связку лука, Детрей очистил две {{выдел|луковицы}}, искрошил, перемешал на большой сковороде с солью|Грин|Джесси и Моргиана|1928|источник=НКРЯ}}~45439~47202 

2  лук~ru~лук~ru~{{=|зелёный лук|перья лука [1] как пищевой продукт}} {{пример|Она тыкала перья {{выдел|лука}} в соль, откусывала хлеб, подносила кружку дрожащей рукой ко рту, в серое пятнышко соли пулями ударялись слёзы и насквозь, до скоблёного дерева, пробивали

In [46]:
for i in ww_def:
  print(i)

лук~ru~лук~ru~{{ботан.|ru}} род луковичных [[растений]] (''Allium'') семейства [[Луковые]] (Alliaceae) {{пример|Там какое-то болото было, а {{выдел|лук}} рос на кочках.|Валентин Распутин|Последний срок|1970|источник=НКРЯ}}~65218~74957
лук~ru~лук~ru~{{=|репчатый лук|[[луковица|луковицы]] лука [1] как пищевой продукт}} {{пример|Кроме того, он был сварен с большим количеством {{выдел|лука}}, а Валя не любила варёный {{выдел|лук}}.|Татьяна Тронина|Русалка для интимных встреч|2004|источник=НКРЯ}}~94921~68014 луковица~ru~луковица~ru~головка лука {{пример|Разыскав связку лука, Детрей очистил две {{выдел|луковицы}}, искрошил, перемешал на большой сковороде с солью|Грин|Джесси и Моргиана|1928|источник=НКРЯ}}~45439~47202
лук~ru~лук~ru~{{=|зелёный лук|перья лука [1] как пищевой продукт}} {{пример|Она тыкала перья {{выдел|лука}} в соль, откусывала хлеб, подносила кружку дрожащей рукой ко рту, в серое пятнышко соли пулями ударялись слёзы и насквозь, до скоблёного дерева, пробивали его, развеивая по

Посмотрим также на гиперонимы

In [38]:
for idx in range(len(synsets)):
    for hypernym in ruwn.get_hypernyms(synsets[idx]):
        print(idx, {w.lemma() for w in hypernym.get_words()})

0 {'овощ'}
1 {'овощ'}
3 {'растение'}
3 {'лилейные'}
4 {'растение'}
5 {'метательное оружие'}
5 {'холодное оружие'}


Как я уже отмечала, значения adagram 0 и 2 относятся к первому выделенному по словарю значению, 1 и 4 - ко второму, а 3 относится к сфере религиозной в качестве имени и не соответсвует ни одному из выделенных по словарю значений. Если посмотреть на толкования и гиперонимы Wiki_ru_wordnet, то видно, что это третье значение adagram-а он тоже не выделил, зато выделил значение ЛУК2 и несколько значений, подходящих для толкования ЛУК1.

Посмотрим, какие значение adagram определит для толкований Wordnet

In [49]:
ww_ada_senses = []
for i in ww_def:
  sense = disambiguate(vm, 'лук', i)
  print(sense)

4
3
3
3
2
2


/usr/local/lib/python3.6/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


In [52]:
voc_mean_sets = []
for i in vocab_meanings:
  s = set(normalize_pm(i))
  voc_mean_sets.append(s)
  print(s)

{'огородный', 'зелёный', 'лука', 'репчатый', 'трубчатый', 'семейство', 'это', 'или', 'луковица', 'лист', 'лилейный', 'овощ', 'съедобный', 'растение'}
{'оружие', 'метание', 'стянуть', 'тетива', 'для', 'лука', 'дуга', 'в', 'стрела', 'из', 'стрелять', 'вид', 'ручной'}


In [53]:
ww_def_sets = []
for i in ww_def:
  s = set(normalize_pm(i))
  ww_def_sets.append(s)
  print(s)

{'alliaceae', 'последний', 'лука', 'allium', 'там', 'семейство', 'нкрить', 'луковый', 'ru', '1970', 'на', 'то', 'ботан', '65218', 'срок', 'кочка', 'какой', 'род', 'распутин', 'расти', 'растение', 'болото', 'выдел', 'источник', 'пример', 'быть', '74957', 'луковичный', 'валентин', 'а'}
{'68014', 'лука', 'с', 'джесси', 'два', '1', 'луковица', 'очистить', 'детрей', '1928', 'тронина', 'нкрить', 'ru', 'валя', 'разыскать', 'русалка', 'пищевой', 'сварить', 'искрошить', 'тот', 'на', 'моргиана', 'перемешать', 'грин', 'количество', 'головка', 'для', 'и', 'больший', 'продукт', 'сковорода', 'татьяна', '2004', '45439', '47202', 'он', 'варёный', 'выдел', 'связка', 'интимный', 'не', 'встреча', 'пример', 'источник', '94921', 'репчатый', 'быть', 'любить', 'кроме', 'соль', 'большой', 'а', 'как'}
{'пятнышко', 'лука', '1998', '3840', 'ударяться', 'в', 'по', 'пуля', '1', 'астафьев', 'нкрить', 'тыкать', 'ru', 'перо', 'она', 'к', 'стол', 'пищевой', 'рука', 'скоблёный', 'мир', 'пыль', 'издание', 'новый', 'и', 

In [59]:
ada_sets = []
for i in range(5):
  s = []
  for j in vm.sense_neighbors('лук', i):
    s.append(j[0])
  ada_sets.append(set(s))
  print(set(s))

/usr/local/lib/python3.6/dist-packages/adagram/model.py:124: RuntimeWarning: invalid value encountered in true_divide
  sim_matrix = np.dot(self.In, s_v) / self.InNorms


{'нарубать', 'петрушка', 'сельдерей', 'зелень', 'нарезать', 'помидор', 'мелко', 'лук-порей', 'нашинковывать', 'морковь'}
{'дротик', 'самострел', 'аркебуза', 'пневматический', 'пращ', 'мушкет', 'стрелять', 'дробовик', 'арбалет', 'аркебуз'}
{'лучок', 'чеснок', 'морковка', 'перец', 'рубленый', 'картошка', 'гриб', 'помидор', 'томат', 'морковь'}
{'евангелист', 'матфей', 'деяние', 'лука', 'иоанн', 'апостол', 'иаков', 'ириней'}
{'тетива', 'самострел', 'лука', 'палица', 'секира', 'стрела', 'копье', 'колчан', 'арбалет'}


In [65]:
inters = []
for i in range(len(ada_sets)):
  for j in range(len(voc_mean_sets)):
    for q in range(len(ww_def_sets)):
      inter = ada_sets[i] & voc_mean_sets[j] & ww_def_sets[q]
      if inter:
        print(i,j,q,inter)

3 0 0 {'лука'}
3 0 1 {'лука'}
3 0 2 {'лука'}
3 0 3 {'лука'}
3 0 4 {'лука'}
3 0 5 {'лука'}
3 1 0 {'лука'}
3 1 1 {'лука'}
3 1 2 {'лука'}
3 1 3 {'лука'}
3 1 4 {'лука'}
3 1 5 {'лука'}
4 0 0 {'лука'}
4 0 1 {'лука'}
4 0 2 {'лука'}
4 0 3 {'лука'}
4 0 4 {'лука'}
4 0 5 {'лука'}
4 1 0 {'лука'}
4 1 1 {'лука'}
4 1 2 {'лука'}
4 1 3 {'лука'}
4 1 4 {'лука'}
4 1 5 {'лука', 'стрела'}


Результаты пересечения толкований не показали хороших результатов. Хотя для значения ЛУК2 индексы значений получились корректными. Для проверки предсказаний adagram я попробую использовать то соотношение значений, которое было выделено ранее.

In [0]:
tested_y0 = []
for item in corpus:
    tmp = {idx: len(set(normalize_pm(item)) &voc_mean_sets[idx]) for idx in range(3)}
    tmp = sorted(tmp.items(), key = lambda x: x[1], reverse=True)
    tested_y0.append(tmp[0][0] + 1)
print(tested_y0)
print(test_y)
print(metrics.f1_score(test_y, tested_y0, average='macro'))

In [0]:
true_labels = []
for i in corpus['label']:
  true_labels.append(i)

In [0]:
k = 0
ada_labels = [] #сделаем поправку на соотношение значений, которое было отмечено ранее.
for i in corpus['predicted_label']:
  k += 1
  if i == 1 or i == 3:
    ada_labels.append(1)
  elif i == 2 or i == 5:
    ada_labels.append(2)
  else:
    ada_labels.append(i)

In [88]:
np.array(true_labels)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [92]:
np.array(ada_labels)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2])

In [95]:
from sklearn import metrics
print(metrics.f1_score(true_labels, ada_labels, average='macro'))

0.983328702417338


Результат получился очень хороший, если соотнести значения adagram со словарными толкованиями так, как было отмечено.